In [190]:
import glob
from pathlib import Path, PurePath
import pandas as pd
import duckdb

# Get all sale files in the specified directory
sale_files = glob.glob(str(Path().resolve().parent / "data/실거래가/*sale*.csv"))

# make dictionary of sale files from the list, with the first part of the filename as the key
sale_files = {PurePath(file).stem.split("_")[0]: file for file in sale_files}
sale_files

{'apart': 'C:\\Users\\USER\\Documents\\quacklytics\\data\\실거래가\\apart_sale_all.csv',
 'commercial': 'C:\\Users\\USER\\Documents\\quacklytics\\data\\실거래가\\commercial_sale_all.csv',
 'detached': 'C:\\Users\\USER\\Documents\\quacklytics\\data\\실거래가\\detached_sale_all.csv',
 'factory': 'C:\\Users\\USER\\Documents\\quacklytics\\data\\실거래가\\factory_sale_all.csv',
 'land': 'C:\\Users\\USER\\Documents\\quacklytics\\data\\실거래가\\land_sale_all.csv',
 'multifamily': 'C:\\Users\\USER\\Documents\\quacklytics\\data\\실거래가\\multifamily_sale_all.csv',
 'officetel': 'C:\\Users\\USER\\Documents\\quacklytics\\data\\실거래가\\officetel_sale_all.csv',
 'presale': 'C:\\Users\\USER\\Documents\\quacklytics\\data\\실거래가\\presale_sale_all.csv'}

In [191]:
list(sale_files.keys())

['apart',
 'commercial',
 'detached',
 'factory',
 'land',
 'multifamily',
 'officetel',
 'presale']

In [192]:
from collections import OrderedDict

korean_keys = OrderedDict(
    [
        ("아파트", "apart"),
        ("연립/다세대", "multifamily"),
        ("단독/다가구", "detached"),
        ("오피스텔", "officetel"),
        # ("분양/입주권", "presale"),
        ("상업/업무용", "commercial"),
        # ("토지", "land"),
        ("공장/창고 등", "factory"),
    ]
)
korean_keys

OrderedDict([('아파트', 'apart'),
             ('연립/다세대', 'multifamily'),
             ('단독/다가구', 'detached'),
             ('오피스텔', 'officetel'),
             ('상업/업무용', 'commercial'),
             ('공장/창고 등', 'factory')])

In [193]:
con = duckdb.connect()

con.execute("""
            INSTALL excel;
            LOAD excel;
            """)

In [194]:
# apart

# read apart csv
# df_apart = pd.read_csv(sale_files['apart'], dtype=str)
apart = con.sql(f"""
    SELECT * REPLACE(CONCAT('아파트', NO) AS NO),
           '아파트' AS type
    FROM read_csv('{sale_files["apart"]}', header=true, all_varchar=true)
""")
apart

┌─────────────┬─────────────────────────────────────┬─────────┬─────────┬─────────┬────────────────────────┬──────────────┬──────────┬─────────┬────────────────┬─────────┬─────────┬─────────┬─────────┬──────────┬────────────────────┬────────────────┬──────────┬──────────────┬──────────┬─────────┐
│     NO      │               시군구                │  번지   │  본번   │  부번   │         단지명         │ 전용면적(㎡) │ 계약년월 │ 계약일  │ 거래금액(만원) │   동    │   층    │ 매수자  │ 매도자  │ 건축년도 │       도로명       │ 해제사유발생일 │ 거래유형 │ 중개사소재지 │ 등기일자 │  type   │
│   varchar   │               varchar               │ varchar │ varchar │ varchar │        varchar         │   varchar    │ varchar  │ varchar │    varchar     │ varchar │ varchar │ varchar │ varchar │ varchar  │      varchar       │    varchar     │ varchar  │   varchar    │ varchar  │ varchar │
├─────────────┼─────────────────────────────────────┼─────────┼─────────┼─────────┼────────────────────────┼──────────────┼──────────┼─────────┼────────────────┼─────────┼─

In [195]:
con.sql("""
        SELECT 계약년월, COUNT(*) as count 
        FROM apart 
        GROUP BY 계약년월 
        ORDER BY count DESC
""")

┌──────────┬────────┐
│ 계약년월 │ count  │
│ varchar  │ int64  │
├──────────┼────────┤
│ 202006   │ 112409 │
│ 202011   │  94480 │
│ 202012   │  87100 │
│ 202002   │  83302 │
│ 202007   │  78316 │
│ 202010   │  72068 │
│ 202005   │  63567 │
│ 202101   │  62491 │
│ 202103   │  62147 │
│ 202105   │  59050 │
│   ·      │    ·   │
│   ·      │    ·   │
│   ·      │    ·   │
│ 202202   │  24466 │
│ 202206   │  23750 │
│ 202201   │  22525 │
│ 202301   │  20209 │
│ 202207   │  18541 │
│ 202208   │  17826 │
│ 202210   │  17228 │
│ 202209   │  16508 │
│ 202211   │  16068 │
│ 202212   │  15642 │
├──────────┴────────┤
│      60 rows      │
│    (20 shown)     │
└───────────────────┘

In [196]:
con.sql("""
        SELECT 동, COUNT(*) as count 
        FROM apart 
        GROUP BY 동 
        ORDER BY count DESC
""")

┌───────────────────────┬─────────┐
│          동           │  count  │
│        varchar        │  int64  │
├───────────────────────┼─────────┤
│ NULL                  │ 1824462 │
│ 101                   │   60995 │
│ 102                   │   52326 │
│ 103                   │   41412 │
│ 104                   │   30786 │
│ 105                   │   28629 │
│ 106                   │   23387 │
│ 107                   │   18609 │
│ 101동                 │   14730 │
│ 108                   │   14532 │
│  ·                    │       · │
│  ·                    │       · │
│  ·                    │       · │
│ 아르미안              │       1 │
│ 길원탑아파트          │       1 │
│ 3434                  │       1 │
│ 삼창아파트            │       1 │
│ 행강타운              │       1 │
│ 도남1차e-편한세상 101 │       1 │
│ 대준블루인            │       1 │
│ 모아센텀스타          │       1 │
│ 연희빌                │       1 │
│ 효림다온              │       1 │
├───────────────────────┴─────────┤
│ 5461 rows (20 shown)  2 columns │
└─

In [197]:
con.sql("""
        SELECT 층, COUNT(*) as count 
        FROM apart 
        GROUP BY 층
        ORDER BY count DESC
""")

┌─────────┬────────┐
│   층    │ count  │
│ varchar │ int64  │
├─────────┼────────┤
│ 5       │ 187341 │
│ 4       │ 187094 │
│ 3       │ 179177 │
│ 2       │ 170952 │
│ 6       │ 151812 │
│ 1       │ 151080 │
│ 7       │ 143006 │
│ 8       │ 140687 │
│ 9       │ 137865 │
│ 10      │ 135884 │
│ ·       │      · │
│ ·       │      · │
│ ·       │      · │
│ 76      │      6 │
│ 72      │      6 │
│ 80      │      6 │
│ 75      │      5 │
│ 79      │      4 │
│ 78      │      4 │
│ 81      │      3 │
│ -3      │      3 │
│ 83      │      2 │
│ 82      │      1 │
├─────────┴────────┤
│     86 rows      │
│    (20 shown)    │
└──────────────────┘

In [198]:
con.sql("""
        SELECT 건축년도, COUNT(*) as count 
        FROM apart 
        GROUP BY 건축년도
        ORDER BY count DESC
""")

┌──────────┬────────┐
│ 건축년도 │ count  │
│ varchar  │ int64  │
├──────────┼────────┤
│ 2018     │ 113698 │
│ 2017     │ 104255 │
│ 1997     │ 103838 │
│ 1998     │  98434 │
│ 1995     │  96320 │
│ 1996     │  94976 │
│ 1994     │  89755 │
│ 1999     │  87902 │
│ 2004     │  85468 │
│ 1993     │  82691 │
│  ·       │     ·  │
│  ·       │     ·  │
│  ·       │     ·  │
│ 1962     │    418 │
│ 1969     │    321 │
│ 1973     │    207 │
│ 1972     │    176 │
│ 1970     │    146 │
│ 1968     │     52 │
│ 1961     │     20 │
│ 1966     │     13 │
│ 1967     │     11 │
│ 1965     │      6 │
├──────────┴────────┤
│      62 rows      │
│    (20 shown)     │
└───────────────────┘

In [199]:
con.sql("""
        SELECT 해제사유발생일, COUNT(*) as count 
        FROM apart 
        GROUP BY 해제사유발생일
        ORDER BY count DESC
""")

┌────────────────┬─────────┐
│ 해제사유발생일 │  count  │
│    varchar     │  int64  │
├────────────────┼─────────┤
│ NULL           │ 2468377 │
│ 20201223       │     447 │
│ 20211022       │     390 │
│ 20200715       │     334 │
│ 20201123       │     326 │
│ 20210813       │     325 │
│ 20200713       │     313 │
│ 20201130       │     312 │
│ 20241207       │     309 │
│ 20200618       │     299 │
│    ·           │       · │
│    ·           │       · │
│    ·           │       · │
│ 20221204       │       1 │
│ 20250506       │       1 │
│ 20240210       │       1 │
│ 20210919       │       1 │
│ 20230604       │       1 │
│ 20240225       │       1 │
│ 20240908       │       1 │
│ 20200426       │       1 │
│ 20230123       │       1 │
│ 20240211       │       1 │
├────────────────┴─────────┤
│   1895 rows (20 shown)   │
└──────────────────────────┘

In [200]:
con.sql("""
        SELECT 거래유형, COUNT(*) as count 
        FROM apart 
        GROUP BY 거래유형
        ORDER BY count DESC
""")

┌──────────┬─────────┐
│ 거래유형 │  count  │
│ varchar  │  int64  │
├──────────┼─────────┤
│ NULL     │ 1402827 │
│ 중개거래 │ 1057548 │
│ 직거래   │  136774 │
└──────────┴─────────┘

In [201]:
con.sql("""
        SELECT 등기일자, COUNT(*) as count 
        FROM apart 
        GROUP BY 등기일자
        ORDER BY count DESC
""")

┌──────────┬─────────┐
│ 등기일자 │  count  │
│ varchar  │  int64  │
├──────────┼─────────┤
│ NULL     │ 1781866 │
│ 24.09.30 │    8181 │
│ 24.08.30 │    7028 │
│ 24.11.29 │    6305 │
│ 24.10.31 │    6081 │
│ 24.06.28 │    5682 │
│ 23.06.30 │    5538 │
│ 23.11.30 │    5097 │
│ 23.04.28 │    4968 │
│ 24.04.30 │    4874 │
│    ·     │       · │
│    ·     │       · │
│    ·     │       · │
│ 25.05.08 │      25 │
│ 25.05.19 │      24 │
│ 25.05.13 │      23 │
│ 23.01.02 │      22 │
│ 25.05.14 │      21 │
│ 25.05.21 │      10 │
│ 25.05.01 │       5 │
│ 23.05.28 │       2 │
│ 25.05.22 │       1 │
│ 23.03.26 │       1 │
├──────────┴─────────┤
│      590 rows      │
│     (20 shown)     │
└────────────────────┘

In [202]:
# multifamily

# read multifamily csv
multifamily = con.sql(f"""
    SELECT * REPLACE(CONCAT('연립/다세대', NO) AS NO),
           '연립/다세대' AS type
    FROM read_csv('{sale_files["multifamily"]}', header=true, all_varchar=true)
""")
multifamily

┌──────────────────┬───────────────────────────────┬─────────┬─────────┬─────────┬──────────────────────────┬──────────────┬────────────────┬──────────┬─────────┬────────────────┬─────────┬─────────┬─────────┬──────────┬─────────────────────┬────────────────┬──────────┬──────────────┬──────────┬─────────────┐
│        NO        │            시군구             │  번지   │  본번   │  부번   │          건물명          │ 전용면적(㎡) │ 대지권면적(㎡) │ 계약년월 │ 계약일  │ 거래금액(만원) │   층    │ 매수자  │ 매도자  │ 건축년도 │       도로명        │ 해제사유발생일 │ 거래유형 │ 중개사소재지 │ 등기일자 │    type     │
│     varchar      │            varchar            │ varchar │ varchar │ varchar │         varchar          │   varchar    │    varchar     │ varchar  │ varchar │    varchar     │ varchar │ varchar │ varchar │ varchar  │       varchar       │    varchar     │ varchar  │   varchar    │ varchar  │   varchar   │
├──────────────────┼───────────────────────────────┼─────────┼─────────┼─────────┼──────────────────────────┼──────────────┼──────────────

In [203]:
con.sql("""
        SELECT 거래유형, COUNT(*) as count 
        FROM multifamily 
        GROUP BY 거래유형
        ORDER BY count DESC
""")

┌──────────┬────────┐
│ 거래유형 │ count  │
│ varchar  │ int64  │
├──────────┼────────┤
│ NULL     │ 374614 │
│ 중개거래 │ 213977 │
│ 직거래   │  96891 │
└──────────┴────────┘

In [204]:
# detached

# read detached csv
detached = con.sql(f"""
    SELECT * REPLACE(CONCAT('단독/다가구', NO) AS NO),
           "주택유형" AS type
    FROM read_csv('{sale_files["detached"]}', header=true, all_varchar=true)
""")
detached

┌─────────────────┬───────────────────────────────────────────┬─────────┬──────────┬──────────┬────────────┬──────────────┬──────────┬─────────┬────────────────┬─────────┬─────────┬──────────┬──────────────────┬────────────────┬──────────┬──────────────┬─────────┐
│       NO        │                  시군구                   │  번지   │ 주택유형 │ 도로조건 │ 연면적(㎡) │ 대지면적(㎡) │ 계약년월 │ 계약일  │ 거래금액(만원) │  매수   │  매도   │ 건축년도 │      도로명      │ 해제사유발생일 │ 거래유형 │ 중개사소재지 │  type   │
│     varchar     │                  varchar                  │ varchar │ varchar  │ varchar  │  varchar   │   varchar    │ varchar  │ varchar │    varchar     │ varchar │ varchar │ varchar  │     varchar      │    varchar     │ varchar  │   varchar    │ varchar │
├─────────────────┼───────────────────────────────────────────┼─────────┼──────────┼──────────┼────────────┼──────────────┼──────────┼─────────┼────────────────┼─────────┼─────────┼──────────┼──────────────────┼────────────────┼──────────┼──────────────┼─────────┤
│ 단

In [205]:
con.sql("""
        SELECT 주택유형, COUNT(*) as count 
        FROM detached 
        GROUP BY 주택유형
        ORDER BY count DESC
""")

┌──────────┬────────┐
│ 주택유형 │ count  │
│ varchar  │ int64  │
├──────────┼────────┤
│ 단독     │ 306439 │
│ 다가구   │  61740 │
└──────────┴────────┘

In [206]:
con.sql("""
        SELECT 거래유형, COUNT(*) as count 
        FROM detached 
        GROUP BY 거래유형
        ORDER BY count DESC
""")

┌──────────┬────────┐
│ 거래유형 │ count  │
│ varchar  │ int64  │
├──────────┼────────┤
│ NULL     │ 197917 │
│ 중개거래 │  99190 │
│ 직거래   │  71072 │
└──────────┴────────┘

In [207]:
# officetel

# read officetel csv
officetel = con.sql(f"""
    SELECT * REPLACE(CONCAT('오피스텔', NO) AS NO),
           '오피스텔' AS type
    FROM read_csv('{sale_files["officetel"]}', header=true, all_varchar=true)
""")
officetel

┌──────────────┬────────────────────────────────┬─────────┬─────────┬─────────┬───────────────────────────────────┬──────────────┬──────────┬─────────┬────────────────┬─────────┬─────────┬─────────┬──────────┬───────────────────┬────────────────┬──────────┬──────────────┬──────────┐
│      NO      │             시군구             │  번지   │  본번   │  부번   │              단지명               │ 전용면적(㎡) │ 계약년월 │ 계약일  │ 거래금액(만원) │   층    │  매수   │  매도   │ 건축년도 │      도로명       │ 해제사유발생일 │ 거래유형 │ 중개사소재지 │   type   │
│   varchar    │            varchar             │ varchar │ varchar │ varchar │              varchar              │   varchar    │ varchar  │ varchar │    varchar     │ varchar │ varchar │ varchar │ varchar  │      varchar      │    varchar     │ varchar  │   varchar    │ varchar  │
├──────────────┼────────────────────────────────┼─────────┼─────────┼─────────┼───────────────────────────────────┼──────────────┼──────────┼─────────┼────────────────┼─────────┼─────────┼─────────┼─────────

In [208]:
con.sql("""
        SELECT 거래유형, COUNT(*) as count 
        FROM officetel 
        GROUP BY 거래유형
        ORDER BY count DESC
""")

┌──────────┬────────┐
│ 거래유형 │ count  │
│ varchar  │ int64  │
├──────────┼────────┤
│ NULL     │ 105658 │
│ 중개거래 │  85392 │
│ 직거래   │  38098 │
└──────────┴────────┘

In [209]:
# commercial

# read commercial csv
commercial = con.sql(f"""
    SELECT * REPLACE(CONCAT('상업/업무용', NO) AS NO),
           "건축물주용도" AS type
    FROM read_csv('{sale_files["commercial"]}', header=true, all_varchar=true, force_not_null=['지분구분'])
""")
commercial

┌─────────────────┬────────────────────────────────────┬─────────┬─────────┬────────────────┬───────────────┬───────────────┬──────────┬─────────────────┬──────────────┬────────────────┬─────────┬─────────┬─────────┬──────────┬─────────┬──────────┬──────────┬────────────────┬──────────┬──────────────┬───────────────┐
│       NO        │               시군구               │  유형   │  지번   │     도로명     │   용도지역    │ 건축물주용도  │ 도로조건 │ 전용/연면적(㎡) │ 대지면적(㎡) │ 거래금액(만원) │   층    │  매수   │  매도   │ 계약년월 │ 계약일  │ 지분구분 │ 건축년도 │ 해제사유발생일 │ 거래유형 │ 중개사소재지 │     type      │
│     varchar     │              varchar               │ varchar │ varchar │    varchar     │    varchar    │    varchar    │ varchar  │     varchar     │   varchar    │    varchar     │ varchar │ varchar │ varchar │ varchar  │ varchar │ varchar  │ varchar  │    varchar     │ varchar  │   varchar    │    varchar    │
├─────────────────┼────────────────────────────────────┼─────────┼─────────┼────────────────┼───────────────┼────────────

In [210]:
con.sql("""
        SELECT 지분구분, COUNT(*) as count 
        FROM commercial 
        GROUP BY 지분구분
        ORDER BY count DESC
""")

┌──────────┬────────┐
│ 지분구분 │ count  │
│ varchar  │ int64  │
├──────────┼────────┤
│ NULL     │ 345200 │
│ 지분     │  44248 │
└──────────┴────────┘

지분구분 공백은 지분 거래가 아니라는 뜻이기 때문에, null이 아님. 코드 수정 통해 반영. 위 표에서는 null이 아닌 빈 문자열로 뜸.

In [211]:
con.sql("""
        SELECT 거래유형, COUNT(*) as count 
        FROM commercial 
        GROUP BY 거래유형
        ORDER BY count DESC
""")

┌──────────┬────────┐
│ 거래유형 │ count  │
│ varchar  │ int64  │
├──────────┼────────┤
│ NULL     │ 178121 │
│ 직거래   │ 111099 │
│ 중개거래 │ 100228 │
└──────────┴────────┘

In [212]:
# factory

# read factory csv
factory = con.sql(f"""
    SELECT * REPLACE(CONCAT('공장/창고 등', NO) AS NO),
           "건축물주용도" AS type
    FROM read_csv('{sale_files["factory"]}', header=true, all_varchar=true, force_not_null=['지분구분'])
""")
factory

┌─────────────────┬─────────────────────────────────────┬─────────┬─────────┬─────────────────┬───────────────┬───────────────────────┬──────────┬─────────────────┬──────────────┬────────────────┬─────────┬─────────┬─────────┬──────────┬─────────┬──────────┬──────────┬────────────────┬──────────┬──────────────┬───────────────────────┐
│       NO        │               시군구                │  유형   │  지번   │     도로명      │   용도지역    │     건축물주용도      │ 도로조건 │ 전용/연면적(㎡) │ 대지면적(㎡) │ 거래금액(만원) │   층    │  매수   │  매도   │ 계약년월 │ 계약일  │ 지분구분 │ 건축년도 │ 해제사유발생일 │ 거래유형 │ 중개사소재지 │         type          │
│     varchar     │               varchar               │ varchar │ varchar │     varchar     │    varchar    │        varchar        │ varchar  │     varchar     │   varchar    │    varchar     │ varchar │ varchar │ varchar │ varchar  │ varchar │ varchar  │ varchar  │    varchar     │ varchar  │   varchar    │        varchar        │
├─────────────────┼─────────────────────────────────────┼─────────┼

In [213]:
con.sql("""
        SELECT 지분구분, COUNT(*) as count 
        FROM factory 
        GROUP BY 지분구분
        ORDER BY count DESC
""")

┌──────────┬───────┐
│ 지분구분 │ count │
│ varchar  │ int64 │
├──────────┼───────┤
│ NULL     │ 75507 │
│ 지분     │  3064 │
└──────────┴───────┘

지분구분 공백은 지분 거래가 아니라는 뜻이기 때문에, null이 아님. 코드 수정 통해 반영. 위 표에서는 null이 아닌 빈 문자열로 뜸.

In [214]:
con.sql("""
        SELECT 거래유형, COUNT(*) as count 
        FROM factory 
        GROUP BY 거래유형
        ORDER BY count DESC
""")

┌──────────┬───────┐
│ 거래유형 │ count │
│ varchar  │ int64 │
├──────────┼───────┤
│ NULL     │ 35181 │
│ 중개거래 │ 26128 │
│ 직거래   │ 17262 │
└──────────┴───────┘

살릴 수 있는 컬럼

아파트:

NO,시군구,번지,단지명,전용면적(㎡),계약년월,계약일,거래금액(만원),층,건축년도,거래유형

연립/다세대:

NO,시군구,번지,건물명,전용면적(㎡),대지권면적(㎡),계약년월,계약일,거래금액(만원),층,건축년도,거래유형

> 대지권 면적: 해당 다세대 건물이 속한 토지 전체 면적 중 해당 세대가 소유하는 토지 지분의 면적
> 
> 전용면적이 해당 구분소유에 대한 면적이라면 필요없을 것.

단독/다가구:

NO,시군구,주택유형,도로조건,연면적(㎡),대지면적(㎡),계약년월,계약일,거래금액(만원),건축년도,거래유형

> 주택유형: 단독 또는 다가구 기재
>
> 주택유형을 그대로 type 에 사용

오피스텔:

NO,시군구,번지,단지명,전용면적(㎡),계약년월,계약일,거래금액(만원),층,건축년도,거래유형

상업/업무용:

NO,시군구,유형,용도지역,건축물주용도,도로조건,전용/연면적(㎡),대지면적(㎡),거래금액(만원),층,계약년월,계약일,지분구분,건축년도,해제사유발생일,거래유형,중개사소재지

> 유형: 일반 또는 집합 기재
>
> 용도지역
>
> 건축물주용도: 업무, 숙박, 제1종근린생활, …
>
> 전용/연면적: 일반건축물과 집합건축물을 아우르기 때문
>
> 지분구분: 지분 거래를 걸러내든지 할 필요가 있어서 데이터에 포함하여야 함.

공장/창고 등:

NO,시군구,유형,용도지역,건축물주용도,도로조건,전용/연면적(㎡),대지면적(㎡),거래금액(만원),층,계약년월,계약일,지분구분,건축년도,거래유형

---

(일부라도) 공통된 컬럼 정리:

NO, 시군구, type, 단지/건물명, 건축물주용도, 계약년월, 계약일, 전용/연면적, 거래금액, 층, 건축년도, 거래유형

> 시군구: 사실 읍면동
>
> 건축물주용도: 아파트, 연립/다세대→공동주택, 단독/다가구→단독주택, 오피스텔→업무 (업무시설 의미)
> 
> type: 실거래가 구분 또는 건축물주용도
>
> 전용/연면적 & 거래금액: 단가로 변환
>
> 층, 거래유형: null인 경우가 많음

공통되지 않았어도 넣어야 하는 컬럼 정리:

지분구분

In [215]:
# 법정동코드, 한국행정구역분류 등

# read bjdong csv
bjdong = con.sql(f"""
    SELECT *,
        substr(법정동코드, 1, 5) AS 시군구코드,
        substr(법정동코드, 6, 5) AS 법정동코드short
    FROM read_csv('{Path().resolve().parent / "data/processed/code_bjd.csv"}', header=true, all_varchar=true, force_not_null=[])
""")
bjdong

┌────────────┬─────────────────────────────┬──────────┬────────────┬─────────────────┐
│ 법정동코드 │          법정동명           │ 폐지여부 │ 시군구코드 │ 법정동코드short │
│  varchar   │           varchar           │ varchar  │  varchar   │     varchar     │
├────────────┼─────────────────────────────┼──────────┼────────────┼─────────────────┤
│ 1100000000 │ 서울특별시                  │ 존재     │ 11000      │ 00000           │
│ 1111000000 │ 서울특별시 종로구           │ 존재     │ 11110      │ 00000           │
│ 1111010100 │ 서울특별시 종로구 청운동    │ 존재     │ 11110      │ 10100           │
│ 1111010200 │ 서울특별시 종로구 신교동    │ 존재     │ 11110      │ 10200           │
│ 1111010300 │ 서울특별시 종로구 궁정동    │ 존재     │ 11110      │ 10300           │
│ 1111010400 │ 서울특별시 종로구 효자동    │ 존재     │ 11110      │ 10400           │
│ 1111010500 │ 서울특별시 종로구 창성동    │ 존재     │ 11110      │ 10500           │
│ 1111010600 │ 서울특별시 종로구 통의동    │ 존재     │ 11110      │ 10600           │
│ 1111010700 │ 서울특별시 종로구 적선동    │ 존재     │ 11110      │ 10700           │


In [216]:
# read KCAD sgg csv
kcad = con.sql(f"""
    SELECT
               시군구코드,
               시도,
               시군구,
               행정구역분류,
               "행정동 영문명칭"
    FROM read_csv('{Path().resolve().parent / "data/processed/code_kcad_sgg_2024.csv"}', header=true, all_varchar=true, force_not_null=[])
""")
kcad

┌────────────┬────────────────┬──────────┬──────────────┬─────────────────┐
│ 시군구코드 │      시도      │  시군구  │ 행정구역분류 │ 행정동 영문명칭 │
│  varchar   │    varchar     │ varchar  │   varchar    │     varchar     │
├────────────┼────────────────┼──────────┼──────────────┼─────────────────┤
│ 11110      │ 서울특별시     │ 종로구   │ 11010        │ Jongno-gu       │
│ 11140      │ 서울특별시     │ 중구     │ 11020        │ Jung-gu         │
│ 11170      │ 서울특별시     │ 용산구   │ 11030        │ Yongsan-gu      │
│ 11200      │ 서울특별시     │ 성동구   │ 11040        │ Seongdong-gu    │
│ 11215      │ 서울특별시     │ 광진구   │ 11050        │ Gwangjin-gu     │
│ 11230      │ 서울특별시     │ 동대문구 │ 11060        │ Dongdaemun-gu   │
│ 11260      │ 서울특별시     │ 중랑구   │ 11070        │ Jungnang-gu     │
│ 11290      │ 서울특별시     │ 성북구   │ 11080        │ Seongbuk-gu     │
│ 11305      │ 서울특별시     │ 강북구   │ 11090        │ Gangbuk-gu      │
│ 11320      │ 서울특별시     │ 도봉구   │ 11100        │ Dobong-gu       │
│   ·        │    ·           │   ·    

In [217]:
bjdong_kcad = con.sql("""
    SELECT 
        b.*,
        k.시도,
        k.시군구,
        k.행정구역분류,
    FROM bjdong b
    JOIN kcad k
    ON b.시군구코드 = k.시군구코드
""")
bjdong_kcad

┌────────────┬───────────────────────────────┬──────────┬────────────┬─────────────────┬────────────┬─────────┬──────────────┐
│ 법정동코드 │           법정동명            │ 폐지여부 │ 시군구코드 │ 법정동코드short │    시도    │ 시군구  │ 행정구역분류 │
│  varchar   │            varchar            │ varchar  │  varchar   │     varchar     │  varchar   │ varchar │   varchar    │
├────────────┼───────────────────────────────┼──────────┼────────────┼─────────────────┼────────────┼─────────┼──────────────┤
│ 1111000000 │ 서울특별시 종로구             │ 존재     │ 11110      │ 00000           │ 서울특별시 │ 종로구  │ 11010        │
│ 1111010100 │ 서울특별시 종로구 청운동      │ 존재     │ 11110      │ 10100           │ 서울특별시 │ 종로구  │ 11010        │
│ 1111010200 │ 서울특별시 종로구 신교동      │ 존재     │ 11110      │ 10200           │ 서울특별시 │ 종로구  │ 11010        │
│ 1111010300 │ 서울특별시 종로구 궁정동      │ 존재     │ 11110      │ 10300           │ 서울특별시 │ 종로구  │ 11010        │
│ 1111010400 │ 서울특별시 종로구 효자동      │ 존재     │ 11110      │ 10400           │ 서울특별시 │ 종로구  │ 11010   

시험용으로 아파트에 left join. 최종적으로는 마스터DB에 left join해서 저장.

In [218]:
apart_joined = con.sql("""
    SELECT
        a.*,
        b.시군구코드,
        b.법정동코드short,
        b.시도,
        b.시군구,
        b.행정구역분류
    FROM apart a
    LEFT JOIN bjdong_kcad b
    ON a.시군구 = b.법정동명
""")
apart_joined

┌─────────────┬─────────────────────────────┬─────────┬─────────┬─────────┬───────────────────────────────┬──────────────┬──────────┬─────────┬────────────────┬─────────┬─────────┬─────────┬─────────┬──────────┬───────────────────┬────────────────┬──────────┬──────────────┬──────────┬─────────┬────────────┬─────────────────┬────────────┬─────────┬──────────────┐
│     NO      │           시군구            │  번지   │  본번   │  부번   │            단지명             │ 전용면적(㎡) │ 계약년월 │ 계약일  │ 거래금액(만원) │   동    │   층    │ 매수자  │ 매도자  │ 건축년도 │      도로명       │ 해제사유발생일 │ 거래유형 │ 중개사소재지 │ 등기일자 │  type   │ 시군구코드 │ 법정동코드short │    시도    │ 시군구  │ 행정구역분류 │
│   varchar   │           varchar           │ varchar │ varchar │ varchar │            varchar            │   varchar    │ varchar  │ varchar │    varchar     │ varchar │ varchar │ varchar │ varchar │ varchar  │      varchar      │    varchar     │ varchar  │   varchar    │ varchar  │ varchar │  varchar   │     varchar     │  varchar   │ varchar │   varch

In [219]:
con.sql("""
        SELECT 법정동코드short, COUNT(*) as count 
        FROM apart_joined 
        GROUP BY 법정동코드short
        ORDER BY count DESC
""")

┌─────────────────┬────────┐
│ 법정동코드short │ count  │
│     varchar     │ int64  │
├─────────────────┼────────┤
│ 10100           │ 167226 │
│ 10300           │ 165053 │
│ 10200           │ 163705 │
│ 10600           │ 134260 │
│ 10500           │ 127707 │
│ 10700           │ 127287 │
│ 10400           │ 125674 │
│ 10800           │ 104052 │
│ 10900           │  91912 │
│ 11000           │  62082 │
│   ·             │      · │
│   ·             │      · │
│   ·             │      · │
│ 43027           │      5 │
│ 25634           │      4 │
│ 25001           │      4 │
│ 37050           │      3 │
│ 16900           │      3 │
│ 26200           │      3 │
│ 43024           │      2 │
│ 41040           │      2 │
│ 40029           │      1 │
│ 43022           │      1 │
├─────────────────┴────────┤
│   354 rows (20 shown)    │
└──────────────────────────┘

## 마스터DB를 만들자

In [220]:
korean_keys.values()

odict_values(['apart', 'multifamily', 'detached', 'officetel', 'commercial', 'factory'])

In [221]:
print(len(apart))
print(len(multifamily))
print(len(detached))
print(len(officetel))
print(len(commercial))
print(len(factory))

2597149
685482
368179
229148
389448
78571


In [222]:
apart2 = con.sql("""
    SELECT
        NO,
        시군구 AS 읍면동,
        '아파트' AS type,
        '공동주택' AS 건축물주용도,
        단지명 AS "단지/건물명",
        계약년월, 
        계약일, 
        "전용면적(㎡)" AS "전용/연면적(㎡)",
        "거래금액(만원)",
        CAST(NULL AS VARCHAR) AS 지분구분,
        층,
        건축년도,
        거래유형
    FROM apart
""")
apart2

┌─────────────┬─────────────────────────────────────┬─────────┬──────────────┬────────────────────────┬──────────┬─────────┬─────────────────┬────────────────┬──────────┬─────────┬──────────┬──────────┐
│     NO      │               읍면동                │  type   │ 건축물주용도 │      단지/건물명       │ 계약년월 │ 계약일  │ 전용/연면적(㎡) │ 거래금액(만원) │ 지분구분 │   층    │ 건축년도 │ 거래유형 │
│   varchar   │               varchar               │ varchar │   varchar    │        varchar         │ varchar  │ varchar │     varchar     │    varchar     │ varchar  │ varchar │ varchar  │ varchar  │
├─────────────┼─────────────────────────────────────┼─────────┼──────────────┼────────────────────────┼──────────┼─────────┼─────────────────┼────────────────┼──────────┼─────────┼──────────┼──────────┤
│ 아파트1     │ 인천광역시 미추홀구 학익동          │ 아파트  │ 공동주택     │ 동아풍림               │ 202001   │ 31      │ 59.7500         │ 25,700         │ NULL     │ 12      │ 1999     │ NULL     │
│ 아파트2     │ 인천광역시 미추홀구 학익동          │ 아파트  │ 공동주택     │ 

In [223]:
multifamily2 = con.sql("""
    SELECT
        NO,
        시군구 AS 읍면동,
        '연립/다세대' AS type,
        '공동주택' AS 건축물주용도,
        건물명 AS "단지/건물명",
        계약년월, 
        계약일, 
        "전용면적(㎡)" AS "전용/연면적(㎡)",
        "거래금액(만원)",
        CAST(NULL AS VARCHAR) AS 지분구분,
        층,
        건축년도,
        거래유형
    FROM multifamily
""")
multifamily2

┌──────────────────┬───────────────────────────────┬─────────────┬──────────────┬──────────────────────────┬──────────┬─────────┬─────────────────┬────────────────┬──────────┬─────────┬──────────┬──────────┐
│        NO        │            읍면동             │    type     │ 건축물주용도 │       단지/건물명        │ 계약년월 │ 계약일  │ 전용/연면적(㎡) │ 거래금액(만원) │ 지분구분 │   층    │ 건축년도 │ 거래유형 │
│     varchar      │            varchar            │   varchar   │   varchar    │         varchar          │ varchar  │ varchar │     varchar     │    varchar     │ varchar  │ varchar │ varchar  │ varchar  │
├──────────────────┼───────────────────────────────┼─────────────┼──────────────┼──────────────────────────┼──────────┼─────────┼─────────────────┼────────────────┼──────────┼─────────┼──────────┼──────────┤
│ 연립/다세대1     │ 대전광역시 대덕구 송촌동      │ 연립/다세대 │ 공동주택     │ 국제빌라(242-53)         │ 202001   │ 31      │ 72.5400         │ 10,300         │ NULL     │ 4       │ 1994     │ NULL     │
│ 연립/다세대2     │ 서울특별시 은평구 신사동      

In [224]:
detached2 = con.sql("""
    SELECT
        NO,
        시군구 AS 읍면동,
        주택유형 AS type,
        '단독주택' AS 건축물주용도,
        CAST(NULL AS VARCHAR) AS "단지/건물명",
        계약년월, 
        계약일, 
        "연면적(㎡)" AS "전용/연면적(㎡)",
        "거래금액(만원)",
        CAST(NULL AS VARCHAR) AS 지분구분,
        CAST(NULL AS VARCHAR) AS 층,
        건축년도,
        거래유형
    FROM detached
""")
detached2

┌─────────────────┬───────────────────────────────────────────┬─────────┬──────────────┬─────────────┬──────────┬─────────┬─────────────────┬────────────────┬──────────┬─────────┬──────────┬──────────┐
│       NO        │                  읍면동                   │  type   │ 건축물주용도 │ 단지/건물명 │ 계약년월 │ 계약일  │ 전용/연면적(㎡) │ 거래금액(만원) │ 지분구분 │   층    │ 건축년도 │ 거래유형 │
│     varchar     │                  varchar                  │ varchar │   varchar    │   varchar   │ varchar  │ varchar │     varchar     │    varchar     │ varchar  │ varchar │ varchar  │ varchar  │
├─────────────────┼───────────────────────────────────────────┼─────────┼──────────────┼─────────────┼──────────┼─────────┼─────────────────┼────────────────┼──────────┼─────────┼──────────┼──────────┤
│ 단독/다가구1    │ 강원특별자치도 춘천시 서면 현암리         │ 단독    │ 단독주택     │ NULL        │ 202001   │ 31      │ 38.25           │ 6,700          │ NULL     │ NULL    │ 2013     │ NULL     │
│ 단독/다가구2    │ 충청남도 아산시 신창면 오목리             │ 단독    │ 단독주택    

In [225]:
officetel2 = con.sql("""
    SELECT
        NO,
        시군구 AS 읍면동,
        '오피스텔' AS type,
        '업무' AS 건축물주용도,  -- 상업/업무용 기재내용에 맞춤
        단지명 AS "단지/건물명",
        계약년월, 
        계약일, 
        "전용면적(㎡)" AS "전용/연면적(㎡)",
        "거래금액(만원)",
        CAST(NULL AS VARCHAR) AS 지분구분,
        층,
        건축년도,
        거래유형
    FROM officetel
""")
officetel2

┌──────────────┬────────────────────────────────┬──────────┬──────────────┬───────────────────────────────────┬──────────┬─────────┬─────────────────┬────────────────┬──────────┬─────────┬──────────┬──────────┐
│      NO      │             읍면동             │   type   │ 건축물주용도 │            단지/건물명            │ 계약년월 │ 계약일  │ 전용/연면적(㎡) │ 거래금액(만원) │ 지분구분 │   층    │ 건축년도 │ 거래유형 │
│   varchar    │            varchar             │ varchar  │   varchar    │              varchar              │ varchar  │ varchar │     varchar     │    varchar     │ varchar  │ varchar │ varchar  │ varchar  │
├──────────────┼────────────────────────────────┼──────────┼──────────────┼───────────────────────────────────┼──────────┼─────────┼─────────────────┼────────────────┼──────────┼─────────┼──────────┼──────────┤
│ 오피스텔1    │ 경기도 성남시 분당구 서현동    │ 오피스텔 │ 업무         │ 분당풍림아이원플러스오피스텔      │ 202001   │ 31      │ 35.0000         │ 15,700         │ NULL     │ 16      │ 2004     │ NULL     │
│ 오피스텔2    │ 경기도 성남시 분당구 서현

In [226]:
commercial2 = con.sql("""
    SELECT
        NO,
        시군구 AS 읍면동,
        건축물주용도 AS type,
        건축물주용도,  -- type과 동일
        CAST(NULL AS VARCHAR) AS "단지/건물명",
        계약년월, 
        계약일, 
        "전용/연면적(㎡)",
        "거래금액(만원)",
        CASE
            WHEN 지분구분 IS NULL OR trim(지분구분) = '' THEN '-'
            ELSE 지분구분
        END AS 지분구분,
        층,
        건축년도,
        거래유형
    FROM commercial
""")
commercial2

┌─────────────────┬────────────────────────────────────┬───────────────┬───────────────┬─────────────┬──────────┬─────────┬─────────────────┬────────────────┬──────────┬─────────┬──────────┬──────────┐
│       NO        │               읍면동               │     type      │ 건축물주용도  │ 단지/건물명 │ 계약년월 │ 계약일  │ 전용/연면적(㎡) │ 거래금액(만원) │ 지분구분 │   층    │ 건축년도 │ 거래유형 │
│     varchar     │              varchar               │    varchar    │    varchar    │   varchar   │ varchar  │ varchar │     varchar     │    varchar     │ varchar  │ varchar │ varchar  │ varchar  │
├─────────────────┼────────────────────────────────────┼───────────────┼───────────────┼─────────────┼──────────┼─────────┼─────────────────┼────────────────┼──────────┼─────────┼──────────┼──────────┤
│ 상업/업무용1    │ 경상남도 진주시 충무공동           │ 업무          │ 업무          │ NULL        │ 202001   │ 31      │ 47.70           │ 8,521          │ -        │ 4       │ 2016     │ NULL     │
│ 상업/업무용2    │ 전북특별자치도 김제시 흥사동       │ 숙박          │ 숙박 

In [227]:
factory2 = con.sql("""
    SELECT
        NO,
        시군구 AS 읍면동,
        건축물주용도 AS type,
        건축물주용도,  -- type과 동일
        CAST(NULL AS VARCHAR) AS "단지/건물명",
        계약년월, 
        계약일, 
        "전용/연면적(㎡)",
        "거래금액(만원)",
        CASE
            WHEN 지분구분 IS NULL OR trim(지분구분) = '' THEN '-'
            ELSE 지분구분
        END AS 지분구분,
        층,
        건축년도,
        거래유형
    FROM factory
""")
factory2

┌─────────────────┬─────────────────────────────────────┬───────────────────────┬───────────────────────┬─────────────┬──────────┬─────────┬─────────────────┬────────────────┬──────────┬─────────┬──────────┬──────────┐
│       NO        │               읍면동                │         type          │     건축물주용도      │ 단지/건물명 │ 계약년월 │ 계약일  │ 전용/연면적(㎡) │ 거래금액(만원) │ 지분구분 │   층    │ 건축년도 │ 거래유형 │
│     varchar     │               varchar               │        varchar        │        varchar        │   varchar   │ varchar  │ varchar │     varchar     │    varchar     │ varchar  │ varchar │ varchar  │ varchar  │
├─────────────────┼─────────────────────────────────────┼───────────────────────┼───────────────────────┼─────────────┼──────────┼─────────┼─────────────────┼────────────────┼──────────┼─────────┼──────────┼──────────┤
│ 공장/창고 등1   │ 서울특별시 금천구 가산동            │ 공장                  │ 공장                  │ NULL        │ 202001   │ 31      │ 287.90          │ 88,158         │ -        │ 9  

In [236]:
unioned = con.sql("""
SELECT * FROM apart2
UNION ALL
SELECT * FROM multifamily2
UNION ALL
SELECT * FROM detached2
UNION ALL
SELECT * FROM officetel2
UNION ALL
SELECT * FROM commercial2
UNION ALL
SELECT * FROM factory2;
""")
realtx_master = con.sql("""
    SELECT
        a.*,
        ROUND(CAST(REPLACE(a."거래금액(만원)", ',', '') AS DOUBLE) / CAST(a."전용/연면적(㎡)" AS DOUBLE), 4) AS "단가_만원/㎡",
        TRIM(
            CASE WHEN TRIM(split_part(a.읍면동, ' ', 1)) != '' THEN split_part(a.읍면동, ' ', 1) ELSE '' END
            || CASE WHEN TRIM(split_part(a.읍면동, ' ', 2)) != '' THEN ' ' || split_part(a.읍면동, ' ', 2) ELSE '' END
            || CASE WHEN TRIM(split_part(a.읍면동, ' ', 3)) != '' THEN ' ' || split_part(a.읍면동, ' ', 3) ELSE '' END
        ) AS 읍면동_앞3단어,
        b.시군구코드,
        b.법정동코드short,
        b.시도,
        b.시군구,
        b.행정구역분류
    FROM unioned a
    LEFT JOIN bjdong_kcad b
    ON TRIM(
            CASE WHEN TRIM(split_part(a.읍면동, ' ', 1)) != '' THEN split_part(a.읍면동, ' ', 1) ELSE '' END
            || CASE WHEN TRIM(split_part(a.읍면동, ' ', 2)) != '' THEN ' ' || split_part(a.읍면동, ' ', 2) ELSE '' END
            || CASE WHEN TRIM(split_part(a.읍면동, ' ', 3)) != '' THEN ' ' || split_part(a.읍면동, ' ', 3) ELSE '' END
        ) = TRIM(b.법정동명)
    ORDER BY NO
""")
realtx_master

┌─────────────────┬─────────────────────────────────────┬─────────────────┬─────────────────┬─────────────┬──────────┬─────────┬─────────────────┬────────────────┬──────────┬─────────┬──────────┬──────────┬──────────────┬────────────────────────────────┬────────────┬─────────────────┬────────────────┬───────────────────┬──────────────┐
│       NO        │               읍면동                │      type       │  건축물주용도   │ 단지/건물명 │ 계약년월 │ 계약일  │ 전용/연면적(㎡) │ 거래금액(만원) │ 지분구분 │   층    │ 건축년도 │ 거래유형 │ 단가_만원/㎡ │         읍면동_앞3단어         │ 시군구코드 │ 법정동코드short │      시도      │      시군구       │ 행정구역분류 │
│     varchar     │               varchar               │     varchar     │     varchar     │   varchar   │ varchar  │ varchar │     varchar     │    varchar     │ varchar  │ varchar │ varchar  │ varchar  │    double    │            varchar             │  varchar   │     varchar     │    varchar     │      varchar      │   varchar    │
├─────────────────┼─────────────────────────────────────┼───────

In [237]:
con.sql("""
        SELECT *
        FROM realtx_master
        WHERE 행정구역분류 IS NULL
""")

┌─────────┬─────────┬─────────┬──────────────┬─────────────┬──────────┬─────────┬─────────────────┬────────────────┬──────────┬─────────┬──────────┬──────────┬──────────────┬────────────────┬────────────┬─────────────────┬─────────┬─────────┬──────────────┐
│   NO    │ 읍면동  │  type   │ 건축물주용도 │ 단지/건물명 │ 계약년월 │ 계약일  │ 전용/연면적(㎡) │ 거래금액(만원) │ 지분구분 │   층    │ 건축년도 │ 거래유형 │ 단가_만원/㎡ │ 읍면동_앞3단어 │ 시군구코드 │ 법정동코드short │  시도   │ 시군구  │ 행정구역분류 │
│ varchar │ varchar │ varchar │   varchar    │   varchar   │ varchar  │ varchar │     varchar     │    varchar     │ varchar  │ varchar │ varchar  │ varchar  │    double    │    varchar     │  varchar   │     varchar     │ varchar │ varchar │   varchar    │
├─────────┴─────────┴─────────┴──────────────┴─────────────┴──────────┴─────────┴─────────────────┴────────────────┴──────────┴─────────┴──────────┴──────────┴──────────────┴────────────────┴────────────┴─────────────────┴─────────┴─────────┴──────────────┤
│                                             

전수 연계 되었음.

오류 유형 정리:

- 실거래가 시군구(읍면동) `세종특별자치시  종촌동` 세종시는 시도이면서 하위 시군구가 없는데, 시도명과 읍면동명 사이 빈칸 2개 있어 연계 오류: CASE WHEN 필요
- 법정동코드 `4119600000,경기도 부천시 오정구 ,존재` 시군구명 뒤 빈칸 존재(다른 구는 안 그럼): TRIM 필요
- 법정동코드 `4729025331,경상북도 경산시 진량읍 평사리(坪沙),존재`, `4729025332,경상북도 경산시 진량읍 평사리(平沙),존재` 평사리가 2개 있고 둘다 현존함. 한자만 다름...
  - https://devtalk.kakao.com/t/topic/85505

In [238]:
con.sql("""
        SELECT *
        FROM unioned
        WHERE 읍면동 LIKE '세종%'
""").limit(5)

┌───────────┬────────────────────────┬─────────┬──────────────┬───────────────────────┬──────────┬─────────┬─────────────────┬────────────────┬──────────┬─────────┬──────────┬──────────┐
│    NO     │         읍면동         │  type   │ 건축물주용도 │      단지/건물명      │ 계약년월 │ 계약일  │ 전용/연면적(㎡) │ 거래금액(만원) │ 지분구분 │   층    │ 건축년도 │ 거래유형 │
│  varchar  │        varchar         │ varchar │   varchar    │        varchar        │ varchar  │ varchar │     varchar     │    varchar     │ varchar  │ varchar │ varchar  │ varchar  │
├───────────┼────────────────────────┼─────────┼──────────────┼───────────────────────┼──────────┼─────────┼─────────────────┼────────────────┼──────────┼─────────┼──────────┼──────────┤
│ 아파트728 │ 세종특별자치시  종촌동 │ 아파트  │ 공동주택     │ 가재마을12단지        │ 202001   │ 31      │ 84.9941         │ 51,500         │ NULL     │ 28      │ 2015     │ NULL     │
│ 아파트729 │ 세종특별자치시  종촌동 │ 아파트  │ 공동주택     │ 가재마을5단지         │ 202001   │ 31      │ 59.9383         │ 32,000         │ NULL     │ 4       

In [239]:
con.sql("""
        SELECT *
        FROM bjdong_kcad
        WHERE 법정동명 == '경기도 부천시 원미구 중동'
""")

┌────────────┬───────────────────────────┬──────────┬────────────┬─────────────────┬─────────┬───────────────┬──────────────┐
│ 법정동코드 │         법정동명          │ 폐지여부 │ 시군구코드 │ 법정동코드short │  시도   │    시군구     │ 행정구역분류 │
│  varchar   │          varchar          │ varchar  │  varchar   │     varchar     │ varchar │    varchar    │   varchar    │
├────────────┼───────────────────────────┼──────────┼────────────┼─────────────────┼─────────┼───────────────┼──────────────┤
│ 4119210800 │ 경기도 부천시 원미구 중동 │ 존재     │ 41192      │ 10800           │ 경기도  │ 부천시 원미구 │ 31051        │
└────────────┴───────────────────────────┴──────────┴────────────┴─────────────────┴─────────┴───────────────┴──────────────┘

In [240]:
con.sql("""
        SELECT *
        FROM bjdong_kcad
        WHERE 법정동명 LIKE '경상북도 경산시 진량읍 평사리%'
""")

┌────────────┬─────────────────────────────────────┬──────────┬────────────┬─────────────────┬──────────┬─────────┬──────────────┐
│ 법정동코드 │              법정동명               │ 폐지여부 │ 시군구코드 │ 법정동코드short │   시도   │ 시군구  │ 행정구역분류 │
│  varchar   │               varchar               │ varchar  │  varchar   │     varchar     │ varchar  │ varchar │   varchar    │
├────────────┼─────────────────────────────────────┼──────────┼────────────┼─────────────────┼──────────┼─────────┼──────────────┤
│ 4729025331 │ 경상북도 경산시 진량읍 평사리(坪沙) │ 존재     │ 47290      │ 25331           │ 경상북도 │ 경산시  │ 37100        │
│ 4729025332 │ 경상북도 경산시 진량읍 평사리(平沙) │ 존재     │ 47290      │ 25332           │ 경상북도 │ 경산시  │ 37100        │
└────────────┴─────────────────────────────────────┴──────────┴────────────┴─────────────────┴──────────┴─────────┴──────────────┘

### 엑셀에 저장

xlsx 파일은 1백만행 제한이 있음. 여러 파일로 나누어 저장

In [241]:
output_to = Path().resolve().parent / "data/실거래가/processed"
output_to.mkdir(parents=True, exist_ok=True)

MAX_ROWS = 1_048_576 - 1  # considering header
table_name = "realtx_master"
output_prefix = "realtx_part"

# 전체 row 수 확인, 파일 수 도출
total_rows = con.sql(f"SELECT COUNT(*) FROM {table_name}").fetchone()[0]  # type: ignore
num_parts = (total_rows + MAX_ROWS - 1) // MAX_ROWS

for i in range(num_parts):
    offset = i * MAX_ROWS  # starts with 0
    output_file = output_to / f"{output_prefix}_{i + 1}.xlsx"  # starts with 1

    con.sql(f"""
        COPY (
            SELECT * FROM {table_name}
            LIMIT {MAX_ROWS} OFFSET {offset}
        ) TO '{output_file}'
        WITH (
            FORMAT xlsx,
            HEADER true,
            SHEET 'data'
        )
    """)
    print(f"Saved {output_file}")

Saved C:\Users\USER\Documents\quacklytics\data\실거래가\processed\realtx_part_1.xlsx
Saved C:\Users\USER\Documents\quacklytics\data\실거래가\processed\realtx_part_2.xlsx
Saved C:\Users\USER\Documents\quacklytics\data\실거래가\processed\realtx_part_3.xlsx
Saved C:\Users\USER\Documents\quacklytics\data\실거래가\processed\realtx_part_4.xlsx
Saved C:\Users\USER\Documents\quacklytics\data\실거래가\processed\realtx_part_5.xlsx


### parquet 파일 저장

In [242]:
output_file = output_to / "realtx.parquet"
con.sql(f"""
    COPY (
        SELECT * FROM {table_name}
    )
    TO '{output_file}'
    WITH (FORMAT PARQUET);
""")
print(f"Saved {output_file}")

Saved C:\Users\USER\Documents\quacklytics\data\실거래가\processed\realtx.parquet
